# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686601


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:04<00:42,  1.57s/it]

Rendering models:  20%|██        | 6/30 [00:04<00:26,  1.11s/it]

Rendering models:  23%|██▎       | 7/30 [00:06<00:26,  1.14s/it]

Rendering models:  40%|████      | 12/30 [00:06<00:14,  1.24it/s]

Rendering models:  50%|█████     | 15/30 [00:06<00:08,  1.73it/s]

Rendering models:  63%|██████▎   | 19/30 [00:06<00:04,  2.42it/s]

Rendering models:  73%|███████▎  | 22/30 [00:06<00:02,  3.31it/s]

Rendering models:  83%|████████▎ | 25/30 [00:06<00:01,  4.51it/s]

SavinSpaceEngineer                    0.000175
mkehrli                               0.000709
thedarktree                           0.000548
kjkavene                              0.000275
shocko61                              0.000414
jmfranci                              0.001431
tosnyder                              0.000471
mbaker23                              0.003501
not-logged-in-fd43696c853d81e3a866    0.035069
FelipedaFonseca                       0.000784
not-logged-in-ad174108f92057f38166    0.048315
clars915                              0.004118
not-logged-in-85b8889f1dc046a97347    0.002650
crush202020                           0.000434
not-logged-in-9f30f411df9d28d2c541    0.002394
acapirala                             0.000257
not-logged-in-04ed9ae880f18762cd9e    0.170847
Lavadude                              0.027394
layne.woodward2701                    0.015138
Windstar187                           0.001484
ShirleyL                              0.000272
onustu       

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())